## EE460J-Lab 2 | September 11, 20
### Can Gokalp (cg39283) | Priyadarshan Patil (pp22352)

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import os

sns.set(style="white", color_codes=True)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

# Programming Questions

## Problem 1

In [ ]:
df1 = pd.read_csv('Lab2_Data/DF1')
df1.head(5)

In [ ]:
df1.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
pd.plotting.scatter_matrix(df1, figsize=(8,6));

In [ ]:
sns.pairplot(df1);

correlated columns: 0-2, 1-3

In [ ]:
corr = df1.corr()

# plot the heatmap
plt.figure(figsize=(10,8)) 
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

In [ ]:
df1_mat = df1.to_numpy()
num_samples = df1.shape[0]
mean = df1_mat.sum(axis=0, keepdims=True)/num_samples
cent = df1_mat - mean
cov = (1.0/(num_samples-1))*np.dot(cent.T, cent)
print('mean: {} \n\n cov: {} \n\n numpy_cov: {}'.format(mean, cov, np.cov(df1_mat, rowvar=False)))

They match since essentially covariance divided by the standard deviations. And so this also shows the correlation except its just harder to interpret as two different variable could be in different scales.

In [ ]:
mean = np.array([0,0,0])
cov = np.matrix([[20, 0, 0],[0, 40, 10],[0, 10, 30]])

ci = 1
cj = 2

def sample(num_samples):
    mvn = np.random.multivariate_normal(mean, cov, num_samples)
    return mvn

est_cov_arr = []
for num_samples in np.linspace(100,10000,num=50):
    est_mean = mvn.sum(axis=0, keepdims=True)/num_samples
    cent = mvn - est_mean
    est_cov = (1.0/(num_samples-1))*np.dot(cent.T, cent)
#     print('mean: {} \n\n cov: {}'.format(est_mean, est_cov))
    est_cov_arr.append(est_cov[ci, cj])

In [ ]:
cov_arr = [cov[ci, cj]]*len(est_cov_arr)

In [ ]:
plt.plot(np.linspace(1000,10000,num=50), est_cov_arr, label='est')
plt.plot(np.linspace(1000,10000,num=50), cov_arr, label='truth')
plt.legend()
plt.xlabel('# Samples')
plt.ylabel('Estimate')

# Problem 2

In [ ]:
df2 = pd.read_csv('Lab2_Data/DF2')
df2.head(5)

In [ ]:
df2.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
plt.scatter(df2['0'].to_numpy(), df2['1'].to_numpy());

Lets look at the covariance matrix to see the variance of the features and the cross feature covariances.

In [ ]:
Z = df2.to_numpy()
Z_cov = np.cov(Z, rowvar=False)
print('covariance of features: \n {}'.format(Z_cov))

We would like to find the directions of the larger spread of data, and values representing the magnitude -- which are the eigenvectors and eigenvalues. If we apply some linear transformation to matrix M (m by m); $Mv_1 = b_1$ we will obtain (m by 1) vector $b_1$. If we select the vector $v_1$ such that after the transformation the length is changed but not the driection $Mv_1 = \lambda v_1$, this vector is called the eigenvector and the $\lambda$ represents the magnitude/eigenvalue (indicating the magnitude of the spread in the direction of the corresponding eigenvector). Lets consider the covariance matrix and find the directions of the spread of the data. 

In [ ]:
e_val, e_vec = np.linalg.eig(Z_cov)
print('eigen_values: {}, \n eigen_vectors: \n {}'.format(e_val, e_vec))

Looking at the eigenvalues we see that much of the variation is along the first eigenvector: [0.707, 0.707] and tiny variation along the second eigenvector. These two vectors are orthogonal to each other - they define a new space. We can project the data points onto this new space.

In [ ]:
projected = Z@e_vec

And now we can plot the transformed data points

In [ ]:
plt.scatter(projected[:,0], projected[:,1])

In this scatter plot we can see that the first data point is more outlying then the second. The spread along that direction is small and so that point looks farther out.

# Problem 3

# Problem 4

In [ ]:
folder_path = 'Names'

def get_file(year=1880):
    year = str(year)
    text_file = os.path.join(folder_path, 'yob' + year + '.txt')
    df = pd.read_csv(text_file, header=None)
    df.rename(columns = {0:'Name', 1:'Sex', 2:'Freq'}, inplace=True)
    return df 

In [ ]:
df = get_file(1880)

In [ ]:
df.head(5)

In [ ]:
df.sort_values(by='Freq', ascending=False)

In [ ]:
def topk(k=5, year=1880):
    df = get_file(year)
    return df.sort_values(by='Freq', ascending=False)[:k-1]['Name'].values

In [ ]:
topk()

In [ ]:
def name_freq(name='John', years=list(np.arange(1880,2016,1))):
    frames = []
    if not isinstance(years,list):
        years = [years]
    for year in years:
        df = get_file(year)
        cur_df = df[df['Name']==name]
        cur_df['Year'] = year
        cur_df['Rel_Freq'] = df['Freq']/df['Freq'].sum()
        cur_df['Pop'] = df['Freq']/df['Freq'].sum()
        frames.append(cur_df)
        yearly_freq = pd.concat(frames)
        yearly_freq.reset_index(drop=True,inplace=True)
    return yearly_freq

In [ ]:
name_freq_df = name_freq()

In [411]:
frames = []
years = np.arange(1880, 2016, 1)
for year in years:
    df = get_file(year)
    df.groupby(['Sex','Name']).sum()
    df['MF_diff'] = df.sort_values(['Sex','Name','Freq']).groupby(['Name'])['Freq'].diff()
    df['Year'] = year
    df.dropna(inplace=True)
    
#     unique_names = df['Name'].unique()
#     for name in unique_names:
#         cur_df = df[df['Name']==name]

#         try:
#             f_freq = cur_df.loc[cur_df['Sex']=='F', 'Freq'].values[0]
#         except:
#             f_freq = 0
            
#         try:
#             m_freq = cur_df.loc[cur_df['Sex']=='M', 'Freq'].values[0] 
#         except:
#             m_freq = 0
#         mf_diff = m_freq - f_freq
        
#         data = [[name, mf_diff, year]] 
#         cur_df = pd.DataFrame(data, columns = ['Name', 'MF_diff', 'Year']) 
#     print(year)
    frames.append(df)
yearly_freq = pd.concat(frames)
yearly_freq.reset_index(drop=True,inplace=True)

In [ ]:
unique_names = yearly_freq['Name'].unique()
for name in unique_names:
    mf_diff = yearly_freq[yearly_freq['Name']==name]['MF_diff'].to_numpy()
    num_flips = ((np.diff(np.sign(mf_diff)) != 0)*1).sum()
    if num_flips > 0:
        print('name: {}'.format(name))

name: Marion
name: Jessie
name: Sidney
name: Leslie
name: Alva
name: Ollie
name: Allie
name: Lynn
name: Jimmie
name: Dee
name: Merle
name: Dell
name: Fay
name: Tommie
name: Lou
name: Carey
name: Augustine
name: Clair
name: Jean
name: Shirley
name: Freddie
name: Lacy
name: Ossie
name: Theo
name: Bird
name: Alpha
name: Erie
name: Oda
name: Sammie
name: Vernie
name: Artie
name: Augusta
name: Cleo
name: Jewel
name: Johnie
name: Byrd
name: Ocie
name: Guadalupe
name: Lue
name: Odie
name: Lonie
name: Mannie
name: Dana
name: Gene
name: Cary
name: Gail
name: Olie
name: Lennie
name: Claudie
name: Donnie
name: Ottie
name: Verne
name: Elza
name: Arlie
name: Bee
name: Orrie
name: Orie
name: Jodie
name: Tracy
name: Osie
name: Golden
name: Valentine
name: Love
name: Laverne
name: Courtney
name: Rene
name: Rennie
name: Billie
name: Pleasant
name: Sydney
name: Oral
name: Hollie
name: Jule
name: Gay
name: Laurie
name: Montie
name: Patsy
name: Otha
name: Ivory
name: Laurel
name: Alvie
name: Woodie
name: 

In [ ]:
yearly_freq[yearly_freq['Name']==Kelsie]['MF_diff'].to_numpy()

In [ ]:
# df_orig = get_file(1880)
# df = df_orig.copy()
# df.groupby(['Sex','Name']).sum()
# df['mf_diff'] = df.sort_values(['Sex','Name','Freq']).groupby(['Name'])['Freq'].diff()
# df.dropna()